In [1]:
import numpy as np
from sklearn import svm,datasets
from sklearn.metrics import roc_curve,auc
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import label_binarize, MultiLabelBinarizer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import SVC
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
df = pd.read_csv('D:/abstract/wos_0531/bib_360446/dropna_abstract_334765_withtopic.csv', engine='python')

In [3]:
df_sim = pd.DataFrame()
df_sim['Abstract'] = df['Abstract']
df_sim['topic'] = df['topic']
df_sim['Research Areas'] = df['Research Areas']

In [4]:
df_sim

,Abstract,topic,Research Areas
0,This paper presents an updated checklist of ma...,13,Marine & Freshwater Biology; Oceanography
1,Small time behavior of gravity driven free sur...,16,Engineering; Oceanography
2,This Collective Article includes records of 29...,13,Fisheries; Geology; Marine & Freshwater Biolog...
3,This Collective Article presents information a...,13,Fisheries; Geology; Marine & Freshwater Biolog...
4,The use of pesticides in agriculture has alway...,73,Engineering; Oceanography
...,...,...,...
334760,The simplicity of the hydrochemical stationari...,40,Engineering; Geology; Water Resources
334761,In many smallholder farms in sub-Saharan Afric...,40,Geology; Meteorology & Atmospheric Sciences; W...
334762,In this study we tested whether we can use a t...,0,Environmental Sciences & Ecology; Remote Sensing
334763,A novel approach of using variability as a maj...,69,Meteorology & Atmospheric Sciences


In [11]:
area_categories = []
for category in list(df['Research Areas']):
    if type(category)!=float:
        cs = category.split(';')
        for c in cs:
            area_categories.append(c.strip())

In [12]:
category = list(set(area_categories))

In [18]:
for i in range(105):
    df_sim[category[i]] = 0

C:\Users\Velaciela\AppData\Local\Temp\ipykernel_12776\1336000183.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_sim[category[i]] = 0
C:\Users\Velaciela\AppData\Local\Temp\ipykernel_12776\1336000183.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_sim[category[i]] = 0
C:\Users\Velaciela\AppData\Local\Temp\ipykernel_12776\1336000183.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at on

In [19]:
df_sim

,Abstract,topic,Research Areas,Thermodynamics,Mechanics,Geography,Astronomy & Astrophysics,Area Studies,Business & Economics,Government & Law,...,Cardiovascular System & Cardiology,History & Philosophy of Science,Evolutionary Biology,Paleontology,Environmental Sciences & Ecology,Electrochemistry,Hematology,Medical Informatics,Metallurgy & Metallurgical Engineering,Meteorology & Atmospheric Sciences
0,This paper presents an updated checklist of ma...,13,Marine & Freshwater Biology; Oceanography,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Small time behavior of gravity driven free sur...,16,Engineering; Oceanography,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,This Collective Article includes records of 29...,13,Fisheries; Geology; Marine & Freshwater Biolog...,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,This Collective Article presents information a...,13,Fisheries; Geology; Marine & Freshwater Biolog...,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,The use of pesticides in agriculture has alway...,73,Engineering; Oceanography,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
334760,The simplicity of the hydrochemical stationari...,40,Engineering; Geology; Water Resources,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
334761,In many smallholder farms in sub-Saharan Afric...,40,Geology; Meteorology & Atmospheric Sciences; W...,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
334762,In this study we tested whether we can use a t...,0,Environmental Sciences & Ecology; Remote Sensing,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
334763,A novel approach of using variability as a maj...,69,Meteorology & Atmospheric Sciences,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [37]:
for i in range(len(df_sim)):
    cate = df_sim.loc[i, 'Research Areas']
    if type(cate)!=float:
        cs = cate.split(';')
        for c in cs:
            df_sim.loc[i, c.strip()] = 1
    else:
        df_sim.loc[i, 'Oceanography'] = 1

In [40]:
df_sim.to_csv('D:/abstract/wos_0531/bib_360446/sim_multilabel.csv', index=False)

In [13]:
x = list(df_sim['Abstract'])

In [14]:
#train_corpus = x[:250000]
#test_corpus = x[250000:]
train_corpus = x
test_corpus = x

In [19]:
label_category = []
for cate_ in list(df_sim['Research Areas']):
    label_c = []
    if type(cate_)!=float:
        cs = cate_.split(';')
        for c in cs:
            label_c.append(c.strip())
    else:
        label_c.append('Oceanography')
    label_category.append(label_c)

In [20]:
label_category

[['Marine & Freshwater Biology', 'Oceanography'],
 ['Engineering', 'Oceanography'],
 ['Fisheries', 'Geology', 'Marine & Freshwater Biology', 'Oceanography'],
 ['Fisheries', 'Geology', 'Marine & Freshwater Biology', 'Oceanography'],
 ['Engineering', 'Oceanography'],
 ['Environmental Sciences & Ecology',
  'Mechanics',
  'Meteorology & Atmospheric Sciences',
  'Oceanography',
  'Water Resources'],
 ['Oceanography', 'Water Resources'],
 ['Engineering', 'Environmental Sciences & Ecology'],
 ['Fisheries', 'Geology', 'Marine & Freshwater Biology', 'Oceanography'],
 ['Environmental Sciences & Ecology', 'Marine & Freshwater Biology'],
 ['Science & Technology - Other Topics'],
 ['Science & Technology - Other Topics'],
 ['Science & Technology - Other Topics', 'Environmental Sciences & Ecology'],
 ['Oceanography'],
 ['Fisheries', 'Geology', 'Marine & Freshwater Biology', 'Oceanography'],
 ['Fisheries', 'Marine & Freshwater Biology', 'Oceanography'],
 ['Physical Geography', 'Geology'],
 ['Environm

In [17]:
label_int = []
for label in label_category:
    label_i = []
    for l in label:
        label_i.append(int(category.index(l)))
    label_int.append(label_i)

In [21]:
label_int

[[24, 100],
 [17, 100],
 [8, 39, 24, 100],
 [8, 39, 24, 100],
 [17, 100],
 [70, 53, 12, 100, 42],
 [100, 42],
 [17, 70],
 [8, 39, 24, 100],
 [70, 24],
 [9],
 [9],
 [9, 70],
 [100],
 [8, 39, 24, 100],
 [8, 24, 100],
 [71, 39],
 [70, 71, 39],
 [8, 39, 24, 100],
 [8, 24, 100],
 [8, 39, 24, 100],
 [24, 100],
 [17, 70],
 [8, 39, 24, 100],
 [8, 39, 24, 100],
 [70, 39, 42],
 [67, 39],
 [17, 70],
 [71, 39],
 [70, 100, 104, 42],
 [70, 39, 42],
 [70, 39, 42],
 [24, 100],
 [100],
 [17, 41],
 [39, 42],
 [24, 100],
 [70, 71, 39],
 [70, 71, 39],
 [70, 71, 39],
 [70, 71, 39],
 [70, 71, 39],
 [70, 71, 39],
 [24, 100],
 [66, 100],
 [71, 39],
 [70, 39],
 [70, 39],
 [87, 79, 86],
 [90],
 [17, 100],
 [100],
 [9, 56],
 [71, 39],
 [66],
 [66],
 [100],
 [24, 100],
 [70, 39, 42],
 [104, 42],
 [17, 100, 68],
 [42],
 [24, 100],
 [17],
 [70, 12],
 [42],
 [42],
 [42],
 [67],
 [42],
 [70, 39, 42],
 [39, 12, 42],
 [17, 70, 42],
 [12],
 [100],
 [17, 100],
 [100, 42],
 [24, 100],
 [87, 100],
 [17, 100],
 [9, 17, 70],

In [22]:
classes = [x for x in range(105)]
mlb = MultiLabelBinarizer(classes=classes)

In [23]:
y_label = mlb.fit_transform(label_int)
#train_label = y_label[:250000]
#test_label = y_label[250000:]
train_label = y_label
test_label = y_label

In [24]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
import tqdm
from sklearn.linear_model import SGDClassifier
from sklearn.datasets import fetch_20newsgroups
from collections import defaultdict
from nltk.tokenize import word_tokenize
import nltk

In [25]:
vectorizer = TfidfVectorizer()
X_train = vectorizer.fit_transform(train_corpus)
X_test = vectorizer.transform(test_corpus)

In [26]:
clf = SGDClassifier()
ovr = OneVsRestClassifier(estimator=clf, n_jobs=-1)

In [27]:
ovr.fit(X_train, train_label)
y_score = ovr.decision_function(X_test)
print("score:", ovr.score(X_test, test_label))

score: 0.2789509058593342


In [45]:
y_pre = ovr.predict(X_test)

In [49]:
y_score_12 = y_score[:,[25,27,36,39,40,43,47,50,71,78,99,104]]
y_score_12

array([[-1.0208455 , -1.40411043, -0.70199252, ..., -0.75453602,
        -1.22840873, -1.0203445 ],
       [-1.00288592, -1.20839824,  0.02172733, ..., -1.27643949,
        -1.20503796, -1.01770402],
       [-1.01123584, -1.44174354, -1.44084813, ..., -0.53481289,
        -1.13216315, -1.02400717],
       ...,
       [-1.00846621, -1.19414587, -1.24400991, ..., -0.85568525,
        -1.2239038 , -1.01207195],
       [-1.00690065,  0.48039063, -1.51084734, ..., -0.98657322,
        -1.36842032, -1.00302499],
       [-0.99995898,  0.28137656, -1.28574664, ..., -1.03837895,
        -1.33260046, -1.01548099]])

In [50]:
test_label_12 = test_label[:,[25,27,36,39,40,43,47,50,71,78,99,104]]
test_label_12

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 1, ..., 0, 0, 0],
       [0, 0, 0, ..., 1, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0]])

In [120]:
y_score_12[0].argmax()

3

In [122]:
test_label_12[0][3]

1

In [55]:
y_score_7 = y_score[:,[9,12,17,24,39,42,66]]
y_score_7

array([[-1.0208455 , -1.40411043, -0.70199252, ..., -0.75453602,
        -1.22840873, -1.0203445 ],
       [-1.00288592, -1.20839824,  0.02172733, ..., -1.27643949,
        -1.20503796, -1.01770402],
       [-1.01123584, -1.44174354, -1.44084813, ..., -0.53481289,
        -1.13216315, -1.02400717],
       ...,
       [-1.00846621, -1.19414587, -1.24400991, ..., -0.85568525,
        -1.2239038 , -1.01207195],
       [-1.00690065,  0.48039063, -1.51084734, ..., -0.98657322,
        -1.36842032, -1.00302499],
       [-0.99995898,  0.28137656, -1.28574664, ..., -1.03837895,
        -1.33260046, -1.01548099]])

In [56]:
test_label_7 = test_label[:,[9,12,17,24,39,42,66]]
test_label_7

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 1, ..., 0, 0, 0],
       [0, 0, 0, ..., 1, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0]])

In [57]:
s = 0
for i in range(len(y_pre)):
    am = y_score_7[i].argmax()
    if test_label_7[i][am] == 1:
        s += 1
s/len(y_pre)

0.5620509909936822

In [28]:
print(category.index('Geology'))
print(category.index('Engineering'))
print(category.index('Marine & Freshwater Biology'))
print(category.index('Environmental Sciences & Ecology'))
print(category.index('Meteorology & Atmospheric Sciences'))
print(category.index('Water Resources'))
print(category.index('Remote Sensing'))
print(category.index('Fisheries'))
print(category.index('Geochemistry & Geophysics'))
print(category.index('Physical Geography'))
print(category.index('Imaging Science & Photographic Technology'))
print(category.index('Science & Technology - Other Topics'))

39
17
24
70
12
42
83
8
66
71
6
9


In [127]:
y_score

array([[-1.01196132, -1.01527208, -1.0095148 , ..., -1.0049037 ,
        -1.00805925, -1.40098402],
       [-1.01000566, -0.99381339, -1.00829864, ..., -1.01224212,
        -1.00900269, -1.20752641],
       [-1.01062915, -1.00945648, -1.00812233, ..., -1.0102275 ,
        -1.00610107, -1.43523059],
       ...,
       [-1.01239835, -1.02474191, -1.01098458, ..., -1.00760769,
        -1.00981792, -1.18347963],
       [-1.01716354, -1.03878563, -1.00922759, ..., -1.01311066,
        -1.00885685,  0.50422218],
       [-1.01739296, -1.03528428, -1.01076175, ..., -1.00554807,
        -1.00927429,  0.30192909]])

In [128]:
score_12 = y_score[:,[25,27,36,39,40,43,47,50,71,78,99,104]]
label_12 = ['Water Resources',
           'Geology',
           'Engineering',
           'Marine & Freshwater Biology',
           'Imaging Science & Photographic Technology',
           'Science & Technology - Other Topics',
           'Remote Sensing',
           'Geochemistry & Geophysics',
           'Fisheries',
           'Physical Geography',
           'Environmental Sciences & Ecology',
           'Meteorology & Atmospheric Sciences']

In [130]:
label=[]
for i in range(len(score_12)):
    label.append(label_12[score_12[i].argmax()])

In [131]:
label

['Marine & Freshwater Biology',
 'Engineering',
 'Marine & Freshwater Biology',
 'Marine & Freshwater Biology',
 'Marine & Freshwater Biology',
 'Water Resources',
 'Water Resources',
 'Marine & Freshwater Biology',
 'Marine & Freshwater Biology',
 'Marine & Freshwater Biology',
 'Geology',
 'Water Resources',
 'Remote Sensing',
 'Geology',
 'Marine & Freshwater Biology',
 'Marine & Freshwater Biology',
 'Geology',
 'Marine & Freshwater Biology',
 'Marine & Freshwater Biology',
 'Marine & Freshwater Biology',
 'Marine & Freshwater Biology',
 'Marine & Freshwater Biology',
 'Marine & Freshwater Biology',
 'Marine & Freshwater Biology',
 'Marine & Freshwater Biology',
 'Geology',
 'Geology',
 'Meteorology & Atmospheric Sciences',
 'Geology',
 'Engineering',
 'Water Resources',
 'Water Resources',
 'Marine & Freshwater Biology',
 'Marine & Freshwater Biology',
 'Engineering',
 'Geology',
 'Marine & Freshwater Biology',
 'Environmental Sciences & Ecology',
 'Geology',
 'Water Resources',
 

In [137]:
label_index=[]
for i in range(len(score_12)):
    label_index.append(score_12[i].argmax())

In [138]:
label_index

[3,
 2,
 3,
 3,
 3,
 0,
 0,
 3,
 3,
 3,
 1,
 0,
 6,
 1,
 3,
 3,
 1,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 1,
 1,
 11,
 1,
 2,
 0,
 0,
 3,
 3,
 2,
 1,
 3,
 10,
 1,
 0,
 10,
 1,
 8,
 5,
 2,
 1,
 3,
 3,
 10,
 2,
 2,
 2,
 2,
 1,
 1,
 11,
 2,
 3,
 0,
 0,
 2,
 0,
 0,
 1,
 11,
 0,
 0,
 0,
 2,
 0,
 1,
 11,
 0,
 11,
 3,
 2,
 3,
 1,
 3,
 2,
 0,
 0,
 2,
 0,
 2,
 1,
 3,
 2,
 3,
 3,
 0,
 0,
 1,
 11,
 2,
 0,
 0,
 3,
 0,
 0,
 3,
 1,
 0,
 0,
 0,
 0,
 0,
 3,
 0,
 0,
 0,
 2,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 11,
 1,
 2,
 6,
 3,
 0,
 0,
 11,
 0,
 0,
 1,
 3,
 3,
 0,
 3,
 6,
 1,
 0,
 3,
 6,
 1,
 1,
 3,
 0,
 10,
 0,
 0,
 0,
 0,
 0,
 2,
 0,
 2,
 3,
 2,
 3,
 0,
 2,
 2,
 0,
 0,
 0,
 0,
 6,
 3,
 2,
 5,
 3,
 11,
 0,
 2,
 3,
 0,
 2,
 2,
 0,
 0,
 2,
 3,
 0,
 3,
 1,
 0,
 2,
 0,
 1,
 0,
 0,
 3,
 1,
 2,
 2,
 3,
 3,
 2,
 0,
 2,
 0,
 0,
 10,
 3,
 0,
 3,
 11,
 0,
 1,
 0,
 3,
 0,
 0,
 10,
 0,
 1,
 0,
 1,
 2,
 1,
 0,
 3,
 0,
 0,
 3,
 0,
 1,
 3,
 2,
 0,
 3,
 0,
 11,
 11,
 3,
 11,
 6,
 3,
 10,
 3,
 3,
 1,
 6,
 2,
 1,
 0,


In [132]:
df_sim['label'] = label

C:\Users\Velaciela\AppData\Local\Temp\ipykernel_12776\562033741.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_sim['label'] = label


In [134]:
df_label = pd.DataFrame()
df_label['Abstract'] = df_sim['Abstract']
df_label['topic'] = df_sim['topic']
df_label['Research Areas'] = df_sim['Research Areas']
df_label['label'] = df_sim['label']

In [139]:
df_label['index'] = label_index

In [140]:
df_label

,Abstract,topic,Research Areas,label,index
0,This paper presents an updated checklist of ma...,13,Marine & Freshwater Biology; Oceanography,Marine & Freshwater Biology,3
1,Small time behavior of gravity driven free sur...,16,Engineering; Oceanography,Engineering,2
2,This Collective Article includes records of 29...,13,Fisheries; Geology; Marine & Freshwater Biolog...,Marine & Freshwater Biology,3
3,This Collective Article presents information a...,13,Fisheries; Geology; Marine & Freshwater Biolog...,Marine & Freshwater Biology,3
4,The use of pesticides in agriculture has alway...,73,Engineering; Oceanography,Marine & Freshwater Biology,3
...,...,...,...,...,...
334760,The simplicity of the hydrochemical stationari...,40,Engineering; Geology; Water Resources,Water Resources,0
334761,In many smallholder farms in sub-Saharan Afric...,40,Geology; Meteorology & Atmospheric Sciences; W...,Water Resources,0
334762,In this study we tested whether we can use a t...,0,Environmental Sciences & Ecology; Remote Sensing,Remote Sensing,6
334763,A novel approach of using variability as a maj...,69,Meteorology & Atmospheric Sciences,Meteorology & Atmospheric Sciences,11


In [142]:
df_label.to_csv('D:/abstract/wos_0531/bib_360446/sim_multilabel_withlabel.csv', index=False)

In [147]:
label_category

[['Marine & Freshwater Biology', 'Oceanography'],
 ['Engineering', 'Oceanography'],
 ['Fisheries', 'Geology', 'Marine & Freshwater Biology', 'Oceanography'],
 ['Fisheries', 'Geology', 'Marine & Freshwater Biology', 'Oceanography'],
 ['Engineering', 'Oceanography'],
 ['Environmental Sciences & Ecology',
  'Mechanics',
  'Meteorology & Atmospheric Sciences',
  'Oceanography',
  'Water Resources'],
 ['Oceanography', 'Water Resources'],
 ['Engineering', 'Environmental Sciences & Ecology'],
 ['Fisheries', 'Geology', 'Marine & Freshwater Biology', 'Oceanography'],
 ['Environmental Sciences & Ecology', 'Marine & Freshwater Biology'],
 ['Science & Technology - Other Topics'],
 ['Science & Technology - Other Topics'],
 ['Science & Technology - Other Topics', 'Environmental Sciences & Ecology'],
 ['Oceanography'],
 ['Fisheries', 'Geology', 'Marine & Freshwater Biology', 'Oceanography'],
 ['Fisheries', 'Marine & Freshwater Biology', 'Oceanography'],
 ['Physical Geography', 'Geology'],
 ['Environm

In [148]:
lc = []

In [149]:
for i in label_category:
    lc_ = []
    for l in i:
        if l != 'Oceanography':
            lc_.append(l)
    lc.append(lc_)

In [151]:
lc_count = []
for i in lc:
    if len(i)==1 and i[0] != 'Oceanography':
        lc_count.append(i[0])

In [152]:
lc_count

['Marine & Freshwater Biology',
 'Engineering',
 'Engineering',
 'Water Resources',
 'Science & Technology - Other Topics',
 'Science & Technology - Other Topics',
 'Marine & Freshwater Biology',
 'Marine & Freshwater Biology',
 'Marine & Freshwater Biology',
 'Marine & Freshwater Biology',
 'Geochemistry & Geophysics',
 'Physics',
 'Engineering',
 'Geochemistry & Geophysics',
 'Geochemistry & Geophysics',
 'Marine & Freshwater Biology',
 'Water Resources',
 'Marine & Freshwater Biology',
 'Engineering',
 'Water Resources',
 'Water Resources',
 'Water Resources',
 'Computer Science',
 'Water Resources',
 'Meteorology & Atmospheric Sciences',
 'Engineering',
 'Water Resources',
 'Marine & Freshwater Biology',
 'Chemistry',
 'Engineering',
 'Geology',
 'Engineering',
 'Marine & Freshwater Biology',
 'Marine & Freshwater Biology',
 'Geology',
 'Geochemistry & Geophysics',
 'Geology',
 'Meteorology & Atmospheric Sciences',
 'Instruments & Instrumentation',
 'Marine & Freshwater Biology',
 

In [153]:
dic_lc = {}
for key in lc_count:
    dic_lc[key]=dic_lc.get(key,0)+1

In [154]:
sorted(dic_lc.items(),key = lambda x:x[1],reverse = True)

[('Engineering', 31663),
 ('Marine & Freshwater Biology', 29063),
 ('Meteorology & Atmospheric Sciences', 28643),
 ('Geology', 25082),
 ('Water Resources', 11063),
 ('Geochemistry & Geophysics', 8560),
 ('Science & Technology - Other Topics', 4917),
 ('Chemistry', 2904),
 ('Environmental Sciences & Ecology', 2231),
 ('Physics', 1665),
 ('Fisheries', 1314),
 ('Optics', 1139),
 ('Acoustics', 863),
 ('Computer Science', 851),
 ('Remote Sensing', 636),
 ('Physical Geography', 526),
 ('Mathematics', 512),
 ('Agriculture', 433),
 ('Energy & Fuels', 265),
 ('Mechanics', 197),
 ('Zoology', 152),
 ('Materials Science', 143),
 ('Automation & Control Systems', 140),
 ('Astronomy & Astrophysics', 122),
 ('Nuclear Science & Technology', 121),
 ('Spectroscopy', 96),
 ('Telecommunications', 85),
 ('Robotics', 62),
 ('Metallurgy & Metallurgical Engineering', 46),
 ('Geography', 36),
 ('Transportation', 34),
 ('Paleontology', 29),
 ('Instruments & Instrumentation', 28),
 ('Business & Economics', 23),
 

In [156]:
for i in label_category:
    if 'Chemistry' in i:
        print(i)

['Chemistry', 'Nuclear Science & Technology', 'Radiology, Nuclear Medicine & Medical Imaging']
['Chemistry', 'Oceanography']
['Chemistry', 'Oceanography']
['Chemistry', 'Nuclear Science & Technology']
['Chemistry', 'Oceanography']
['Chemistry', 'Spectroscopy']
['Chemistry', 'Engineering', 'Materials Science', 'Physics']
['Chemistry', 'Oceanography']
['Chemistry', 'Nuclear Science & Technology']
['Chemistry', 'Engineering', 'Instruments & Instrumentation']
['Chemistry', 'Nuclear Science & Technology', 'Radiology, Nuclear Medicine & Medical Imaging']
['Chemistry', 'Instruments & Instrumentation']
['Chemistry', 'Instruments & Instrumentation']
['Chemistry', 'Nuclear Science & Technology', 'Radiology, Nuclear Medicine & Medical Imaging']
['Chemistry', 'Nuclear Science & Technology']
['Chemistry', 'Oceanography']
['Chemistry', 'Oceanography']
['Chemistry', 'Science & Technology - Other Topics']
['Chemistry', 'Oceanography']
['Chemistry', 'Engineering', 'Materials Science', 'Physics']
['Chem

['Chemistry', 'Physics']
['Chemistry', 'Environmental Sciences & Ecology', 'Instruments & Instrumentation']
['Chemistry', 'Physics']
['Chemistry', 'Physics']
['Chemistry', 'Nuclear Science & Technology', 'Radiology, Nuclear Medicine & Medical Imaging']
['Chemistry', 'Engineering', 'Instruments & Instrumentation']
['Chemistry', 'Engineering', 'Instruments & Instrumentation']
['Chemistry', 'Engineering', 'Instruments & Instrumentation']
['Chemistry', 'Engineering', 'Materials Science', 'Physics']
['Chemistry', 'Engineering', 'Materials Science', 'Physics']
['Chemistry', 'Engineering', 'Materials Science', 'Physics']
['Chemistry', 'Engineering', 'Instruments & Instrumentation']
['Chemistry', 'Engineering', 'Materials Science', 'Physics']
['Chemistry', 'Engineering', 'Instruments & Instrumentation']
['Chemistry', 'Engineering', 'Materials Science', 'Physics']
['Chemistry', 'Environmental Sciences & Ecology']
['Chemistry', 'Materials Science', 'Metallurgy & Metallurgical Engineering', 'Phys

In [157]:
# 独特标签
print(category.index('Engineering'))
print(category.index('Marine & Freshwater Biology'))
print(category.index('Meteorology & Atmospheric Sciences'))
print(category.index('Geology'))
print(category.index('Water Resources'))
print(category.index('Geochemistry & Geophysics'))
print(category.index('Science & Technology - Other Topics'))

36
39
104
27
25
50
43


In [159]:
score_7 = y_score[:,[25,27,36,39,43,50,104]]
label_7 = ['Water Resources',
           'Geology',
           'Engineering',
           'Marine & Freshwater Biology',
           'Science & Technology - Other Topics',
           'Geochemistry & Geophysics',
           'Meteorology & Atmospheric Sciences']

In [160]:
unique_label=[]
for i in range(len(score_7)):
    unique_label.append(label_7[score_7[i].argmax()])

In [161]:
unique_label

['Marine & Freshwater Biology',
 'Engineering',
 'Marine & Freshwater Biology',
 'Marine & Freshwater Biology',
 'Marine & Freshwater Biology',
 'Water Resources',
 'Water Resources',
 'Marine & Freshwater Biology',
 'Marine & Freshwater Biology',
 'Marine & Freshwater Biology',
 'Geology',
 'Water Resources',
 'Water Resources',
 'Geology',
 'Marine & Freshwater Biology',
 'Marine & Freshwater Biology',
 'Geology',
 'Marine & Freshwater Biology',
 'Marine & Freshwater Biology',
 'Marine & Freshwater Biology',
 'Marine & Freshwater Biology',
 'Marine & Freshwater Biology',
 'Marine & Freshwater Biology',
 'Marine & Freshwater Biology',
 'Marine & Freshwater Biology',
 'Geology',
 'Geology',
 'Meteorology & Atmospheric Sciences',
 'Geology',
 'Engineering',
 'Water Resources',
 'Water Resources',
 'Marine & Freshwater Biology',
 'Marine & Freshwater Biology',
 'Engineering',
 'Geology',
 'Marine & Freshwater Biology',
 'Science & Technology - Other Topics',
 'Geology',
 'Water Resources

In [162]:
unique_label_index=[]
for i in range(len(score_7)):
    unique_label_index.append(score_7[i].argmax())

In [163]:
df_label['unique_index'] = unique_label_index

In [164]:
df_label.to_csv('D:/abstract/wos_0531/bib_360446/sim_multilabel_withlabel.csv', index=False)

In [29]:
# 是否在原标签中
print(category.index('Engineering'))
print(category.index('Marine & Freshwater Biology'))
print(category.index('Meteorology & Atmospheric Sciences'))
print(category.index('Geology'))
print(category.index('Water Resources'))
print(category.index('Geochemistry & Geophysics'))
print(category.index('Science & Technology - Other Topics'))

17
24
12
39
42
66
9


In [30]:
score_7 = y_score[:,[9,12,17,24,39,42,66]]
label_7 = ['Science & Technology - Other Topics',
           'Meteorology & Atmospheric Sciences',
           'Engineering',
           'Marine & Freshwater Biology',
           'Geology',
           'Water Resources',
           'Geochemistry & Geophysics']

In [31]:
score_7

array([[-1.0208455 , -1.40411043, -0.70199252, ..., -0.75453602,
        -1.22840873, -1.0203445 ],
       [-1.00288592, -1.20839824,  0.02172733, ..., -1.27643949,
        -1.20503796, -1.01770402],
       [-1.01123584, -1.44174354, -1.44084813, ..., -0.53481289,
        -1.13216315, -1.02400717],
       ...,
       [-1.00846621, -1.19414587, -1.24400991, ..., -0.85568525,
        -1.2239038 , -1.01207195],
       [-1.00690065,  0.48039063, -1.51084734, ..., -0.98657322,
        -1.36842032, -1.00302499],
       [-0.99995898,  0.28137656, -1.28574664, ..., -1.03837895,
        -1.33260046, -1.01548099]])

In [34]:
test_7 = test_label[:,[9,12,17,24,39,42,66]]

In [44]:
test_7

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 1, ..., 0, 0, 0],
       [0, 0, 0, ..., 1, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0]])

In [93]:
m = 0
for i in range(len(test_7)):
    if 1 in test_7[i]:
        m += 1
m/len(test_7)

0.763404179050976

In [88]:
ifin_label=[]
ifin_label_index=[]
for i in range(len(score_7)):
    if 1 in test_7[i]:
        arg_1 = [i for i, x in enumerate(test_7[i]) if x == 1]
        arg_1_max = score_7[i][arg_1].argmax()
        arg_max = arg_1[arg_1_max]
        ifin_label_index.append(arg_max)
        ifin_label.append(label_7[arg_max])
    else:
        arg_max = score_7[i].argmax()
        ifin_label_index.append(arg_max)
        ifin_label.append(label_7[arg_max])

In [89]:
ifin_label

['Marine & Freshwater Biology',
 'Engineering',
 'Marine & Freshwater Biology',
 'Marine & Freshwater Biology',
 'Engineering',
 'Water Resources',
 'Water Resources',
 'Engineering',
 'Marine & Freshwater Biology',
 'Marine & Freshwater Biology',
 'Science & Technology - Other Topics',
 'Science & Technology - Other Topics',
 'Science & Technology - Other Topics',
 'Geology',
 'Marine & Freshwater Biology',
 'Marine & Freshwater Biology',
 'Geology',
 'Geology',
 'Marine & Freshwater Biology',
 'Marine & Freshwater Biology',
 'Marine & Freshwater Biology',
 'Marine & Freshwater Biology',
 'Engineering',
 'Marine & Freshwater Biology',
 'Marine & Freshwater Biology',
 'Geology',
 'Geology',
 'Engineering',
 'Geology',
 'Water Resources',
 'Water Resources',
 'Water Resources',
 'Marine & Freshwater Biology',
 'Marine & Freshwater Biology',
 'Engineering',
 'Geology',
 'Marine & Freshwater Biology',
 'Geology',
 'Geology',
 'Geology',
 'Geology',
 'Geology',
 'Geology',
 'Marine & Fresh

In [91]:
s = 0
for i in range(len(y_pre)):
    am = ifin_label_index[i]
    if test_label_7[i][am] == 1:
        s += 1
s/len(y_pre)

0.763404179050976

In [95]:
df_sim

,Abstract,topic,Research Areas
0,This paper presents an updated checklist of ma...,13,Marine & Freshwater Biology; Oceanography
1,Small time behavior of gravity driven free sur...,16,Engineering; Oceanography
2,This Collective Article includes records of 29...,13,Fisheries; Geology; Marine & Freshwater Biolog...
3,This Collective Article presents information a...,13,Fisheries; Geology; Marine & Freshwater Biolog...
4,The use of pesticides in agriculture has alway...,73,Engineering; Oceanography
...,...,...,...
334760,The simplicity of the hydrochemical stationari...,40,Engineering; Geology; Water Resources
334761,In many smallholder farms in sub-Saharan Afric...,40,Geology; Meteorology & Atmospheric Sciences; W...
334762,In this study we tested whether we can use a t...,0,Environmental Sciences & Ecology; Remote Sensing
334763,A novel approach of using variability as a maj...,69,Meteorology & Atmospheric Sciences


In [96]:
df_sim['label'] = ifin_label
df_sim['index'] = ifin_label_index

In [97]:
df_sim

,Abstract,topic,Research Areas,label,index
0,This paper presents an updated checklist of ma...,13,Marine & Freshwater Biology; Oceanography,Marine & Freshwater Biology,3
1,Small time behavior of gravity driven free sur...,16,Engineering; Oceanography,Engineering,2
2,This Collective Article includes records of 29...,13,Fisheries; Geology; Marine & Freshwater Biolog...,Marine & Freshwater Biology,3
3,This Collective Article presents information a...,13,Fisheries; Geology; Marine & Freshwater Biolog...,Marine & Freshwater Biology,3
4,The use of pesticides in agriculture has alway...,73,Engineering; Oceanography,Engineering,2
...,...,...,...,...,...
334760,The simplicity of the hydrochemical stationari...,40,Engineering; Geology; Water Resources,Water Resources,5
334761,In many smallholder farms in sub-Saharan Afric...,40,Geology; Meteorology & Atmospheric Sciences; W...,Water Resources,5
334762,In this study we tested whether we can use a t...,0,Environmental Sciences & Ecology; Remote Sensing,Geology,4
334763,A novel approach of using variability as a maj...,69,Meteorology & Atmospheric Sciences,Meteorology & Atmospheric Sciences,1


In [98]:
df_sim.to_csv('D:/abstract/wos_0531/bib_360446/unique_multilabel_withlabel.csv', index=False)

In [99]:
df_sim

,Abstract,topic,Research Areas,label,index
0,This paper presents an updated checklist of ma...,13,Marine & Freshwater Biology; Oceanography,Marine & Freshwater Biology,3
1,Small time behavior of gravity driven free sur...,16,Engineering; Oceanography,Engineering,2
2,This Collective Article includes records of 29...,13,Fisheries; Geology; Marine & Freshwater Biolog...,Marine & Freshwater Biology,3
3,This Collective Article presents information a...,13,Fisheries; Geology; Marine & Freshwater Biolog...,Marine & Freshwater Biology,3
4,The use of pesticides in agriculture has alway...,73,Engineering; Oceanography,Engineering,2
...,...,...,...,...,...
334760,The simplicity of the hydrochemical stationari...,40,Engineering; Geology; Water Resources,Water Resources,5
334761,In many smallholder farms in sub-Saharan Afric...,40,Geology; Meteorology & Atmospheric Sciences; W...,Water Resources,5
334762,In this study we tested whether we can use a t...,0,Environmental Sciences & Ecology; Remote Sensing,Geology,4
334763,A novel approach of using variability as a maj...,69,Meteorology & Atmospheric Sciences,Meteorology & Atmospheric Sciences,1
